In [1]:
import numpy as np
import pandas as pd
import importlib
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter('ignore')

In [4]:
df = pd.read_csv('all.csv', index_col = 0)

In [6]:
df_y = df[[col for col in df.columns if 'y_' in col]]

In [12]:
df_x = df.drop([col for col in df.columns if 'y_' in col], axis = 1)

In [13]:
df_x = df_x.drop(['make', 'op_descriptions', 'date', 'fuel', 'appointment_month', 'season', 'year'], axis = 1)